In [96]:
import pandas as pd
import swifter

In [2]:
otu_data = pd.read_csv("../data/ecam-table-taxa.tsv",
                       header=1, sep='\t')
otu_data = otu_data.set_index('feature-id')

In [3]:
meta_data = pd.read_csv("../data/ecam-sample-metadata.tsv", sep='\t')
meta_data = meta_data[1:]
meta_data = meta_data.rename(columns={'#SampleID': 'Sample.ID'})

In [314]:
# ANCOM imports
import numpy as np
from scipy.stats import norm, mannwhitneyu, kruskal
import statsmodels.api as sm
from statsmodels.formula.api import ols, mixedlm
from statsmodels.stats.multitest import multipletests

In [5]:
def _outlier_check(x, out_cut):
    # Fitting the mixture model using the algorithm of Peddada, S. Das, and JT Gene Hwang (2002)
    mu1, mu2 = np.nanquantile(x, (0.25, 0.75))
    sigma1 = mu2 - mu1
    sigma2 = sigma1
    pi = 0.75
    n = len(x)
    epsilon = 100
    tol = 1e-5
    score = pi * norm.pdf(x, mu1, sigma1) / \
        ((1 - pi) * norm.pdf(x, mu2, sigma2))
    while epsilon > tol:
        grp1_ind = score >= 1
        not_grp1_ind = np.logical_not(grp1_ind)
        mu1_new = x[grp1_ind].mean()
        mu2_new = x[not_grp1_ind].mean()
        sigma1_new = x[grp1_ind].std()
        if np.isnan(sigma1_new):
            sigma1_new = 0.
        sigma2_new = x[not_grp1_ind].std()
        if np.isnan(sigma2_new):
            sigma2_new = 0.
        pi_new = sum(grp1_ind) / n
        
        para = [mu1_new, mu2_new, sigma1_new, sigma2_new, pi_new]
        if np.any(np.isnan(para)):
            break
        
        if sigma1_new == 0.:
            pdf1 = np.zeros_like(x)
            pdf1[x == mu1_new] = np.inf
        else:
            pdf1 = norm.pdf(x, mu1_new, sigma1_new)
        if sigma2_new == 0.:
            pdf2 = np.zeros_like(x)
            pdf2[x == mu2_new] = np.inf
        else:
            pdf2 = norm.pdf(x, mu2_new, sigma2_new)
        score = pi_new * pdf1 / ((1 - pi_new) * pdf2)
        
        old = np.array([mu1, mu2, sigma1, sigma2, pi])
        epsilon = np.linalg.norm(old - para)
        mu1, mu2, sigma1, sigma2, pi = para

    if mu1 + 1.96 * sigma1 < mu2 - 1.96 * sigma2:
        if pi < out_cut:
            return pd.Series(grp1_ind)
        elif pi > 1 - out_cut:
            return pd.Series(not_grp1_ind)
    return pd.Series([False]*n)
    

def identify_outliers(feature_table, meta_data, group_var, out_cut):
    z = feature_table + 1 # Add pseudo-count (1) # EEEE is this ok if data isn't counts?
    f = z.apply(np.log)
    f[f == 0] = np.nan # EEEE [sic]
    f = f.mean(axis=0, skipna=True)
    groups = meta_data[group_var]
    groups.index = feature_table.columns
    group_means = f.groupby(groups).mean()
    notna_groups = pd.notna(groups)
    group_means = group_means[groups[notna_groups].values]
    group_means.index = groups[notna_groups].index
    e = pd.Series([0]*f.size, index=f.index)
    e[notna_groups] = f[notna_groups] - group_means
    y = z - e

    def row_outlier_check(row):
        return row.groupby(groups).apply(
            lambda x: _outlier_check(x, out_cut))
    out_ind = y.apply(row_outlier_check, axis=1)
    return np.array(out_ind)


def identify_structure_zeros(
    feature_table, meta_data, group_var, neg_lb):
    group = meta_data[group_var]
    group.index = feature_table.columns
    present_table = feature_table.copy()
    present_table[present_table.isna()] = 0
    present_table[present_table != 0] = 1

    p_hat = present_table.apply(
        lambda row: row.groupby(group).mean(), axis=1)
    samp_size = feature_table.apply(
        lambda row: row.notna().groupby(group).sum(), axis=1)
    p_hat_lo = p_hat - 1.96*np.sqrt(p_hat*(1 - p_hat)/samp_size)
    
    struc_zero = (p_hat == 0)*1
    # Whether we need to classify a taxon into structural zero by its negative lower bound?
    if neg_lb:
        struc_zero[p_hat_lo <= 0] = 1
    
    # Entries considered to be structural zeros are set to be 0s
    struc_ind = struc_zero[group]
    struc_ind.columns = feature_table.columns
    
    struc_zero.columns = [
            'structural_zero (' + c + ')' for c in struc_zero.columns]
    
    return struc_ind, struc_zero

def feature_table_pre_process(
    feature_table, meta_data, sample_var, group_var=None,
    out_cut=0.05, zero_cut=0.9, lib_cut=1000, neg_lb=True
):
    # OTU table should be a pandas.DataFrame with each feature in rows and sample in columns.
    # Metadata should be a pandas.DataFrame containing the sample identifier.
    
    # Drop unused levels
    # meta_data[] = lapply(meta_data, function(x) if(is.factor(x)) factor(x) else x) # EEEE is this step necessary? Assuming "no" for now
    
    # Match sample IDs between metadata and feature table
    sample_ID = meta_data[sample_var]
    sample_ID = sample_ID[sample_ID.isin(feature_table.columns)]
    feature_table = feature_table[sample_ID]
    meta_data = meta_data[meta_data[sample_var].isin(sample_ID)]
    
    # 1. Identify outliers within each taxon
    if (group_var is not None):
        out_ind = identify_outliers(
            feature_table, meta_data, group_var, out_cut)
        feature_table[out_ind] = np.nan

    # 2. Discard taxa with zeros  >=  zero_cut
    zero_prop = feature_table.apply(
        lambda x: (x==0).sum() / x.notna().sum(), axis=1)
    feature_table = feature_table[zero_prop < zero_cut]
    
    # 3. Discard samples with library size < lib_cut
    lib_size = feature_table.sum(axis=0, skipna=True)
    feature_table = feature_table[
        feature_table.columns[lib_size >= lib_cut]]
    meta_data = meta_data[(lib_size >= lib_cut).values]

    
    # 4. Identify taxa with structure zeros
    if (group_var is not None):
        struc_ind, struc_zero = identify_structure_zeros(
            feature_table, meta_data, group_var, neg_lb)
        feature_table = feature_table * (1 - struc_ind)
    else:
        struc_zero = None
        
    return dict(
        feature_table = feature_table,
        meta_data = meta_data,
        structure_zeros = struc_zero
    )

In [6]:
def test_identify_outliers():
    feature_table = pd.read_csv("../data/ecam-table-taxa.tsv",
                           header=1, sep='\t')
    feature_table = feature_table.set_index('feature-id')
    meta_data = pd.read_csv(
        "../data/ecam-sample-metadata.tsv", sep='\t')
    meta_data = meta_data[1:]
    meta_data = meta_data.rename(columns={'#SampleID': 'Sample.ID'})
    
    sample_ID = meta_data['Sample.ID']
    sample_ID = sample_ID[sample_ID.isin(feature_table.columns)]
    feature_table = feature_table[sample_ID]
    meta_data = meta_data[meta_data['Sample.ID'].isin(sample_ID)]
    
    out_ind = identify_outliers(
        feature_table, meta_data, 'delivery', 0.05)
    answer = pd.read_csv('outliers.tsv', sep=' ', header=None)
    assert np.all(out_ind == answer)


def test_identify_structure_zeros():
    feature_table = pd.read_csv("../data/ecam-table-taxa.tsv",
                           header=1, sep='\t')
    feature_table = feature_table.set_index('feature-id')
    meta_data = pd.read_csv(
        "../data/ecam-sample-metadata.tsv", sep='\t')
    meta_data = meta_data[1:]
    meta_data = meta_data.rename(columns={'#SampleID': 'Sample.ID'})
    
    sample_ID = meta_data['Sample.ID']
    sample_ID = sample_ID[sample_ID.isin(feature_table.columns)]
    feature_table = feature_table[sample_ID]
    meta_data = meta_data[meta_data['Sample.ID'].isin(sample_ID)]
    
    feature_table = pd.read_csv('feature-table.tsv', sep=' ')
    struc_ind, struc_zero = identify_structure_zeros(
            feature_table, meta_data, 'day_of_life', True)
    answer = pd.read_csv('struc-zero.tsv', sep=' ')
    answer.columns = [
            'structural_zero (' + c + ')' for c in answer.columns]
    assert np.all(struc_zero == answer)


def test_feature_table_pre_process():
    otu_data = pd.read_csv("../data/ecam-table-taxa.tsv",
                       header=1, sep='\t')
    otu_data = otu_data.set_index('feature-id')
    
    meta_data = pd.read_csv("../data/ecam-sample-metadata.tsv", sep='\t')
    meta_data = meta_data[1:]
    meta_data = meta_data.rename(columns={'#SampleID': 'Sample.ID'})
    
    result = feature_table_pre_process(
        otu_data, meta_data, 'Sample.ID', 'delivery',
        out_cut=0.05, zero_cut=0.9, lib_cut=0, neg_lb=True)
    
    answer = pd.read_csv('delivery-feature-table.tsv', sep=' ')
    assert np.allclose(result['feature_table'], answer, equal_nan=True)
    assert np.all(result['structure_zeros'] == 0)
    assert np.all(meta_data == result['meta_data'])

In [7]:
test_identify_outliers()
test_identify_structure_zeros()
test_feature_table_pre_process()

/home/ben/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/home/ben/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: overflow encountered in true_divide
/home/ben/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
/home/ben/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in greater_equal
  del sys.path[0]
/home/ben/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/home/ben/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: overflow encountered in true_divide
/home/ben/minicond

In [346]:
def _p_function(main_var, meta_data, adj_formula, mixedlm_kwargs):
    # Get a function to calcuate a p-value for each row using 
    # the appropriate model
    if mixedlm_kwargs is None and adj_formula is None:
        # Basic model
        factors = set(meta_data[main_var])
        indexes = [meta_data[main_var] == f for f in factors]
        if len(factors) == 2:
            # Two levels: Wilcoxon rank-sum test
            def manwhittneyup(row):
                # x = [row[ix[row.index]].dropna() for ix in indexes] this makes this work with swifter, but then swifter does nothing, so whatever
                x = [row[ix].dropna() for ix in indexes]
                u, p = mannwhitneyu(*x, alternative='two-sided')
                return p
            return manwhittneyup
        # More than two levels: Kruskal-Wallis test
        def kruskalp(row): # could be parallelised
            x = [row[ix] for ix in indexes]
            k, p = kruskal(*x, nan_policy='omit')
            return p
        return kruskalp
    elif mixedlm_kwargs is None and adj_formula is not None:
        # Model: ANOVA
        formula = 'x ~ ' + main_var + ' + ' + adj_formula
        def anovap(row):
            row.name = 'x'
            data = meta_data.join(row)
            lm = ols(formula, data=data, missing='drop').fit()
            anova = sm.stats.anova_lm(lm, typ=1) # to match R implemetation, consider changing to typ 2
            return anova['PR(>F)'][main_var]
        return anovap
    # Model: Mixed-effects model
    formula = 'x ~ ' + main_var
    if adj_formula is not None:
        formula += ' + ' + adj_formula
    def mixedlmp(row):
        row.name = 'x'
        data = meta_data.join(row)
        # print(formula)
        # print(data)
        # print(mixedlm_kwargs)
        mdf = mixedlm(
            formula, data, missing='drop', **mixedlm_kwargs).fit()
        # print(mdf.summary())
        # print(mdf.params.index)
        f_test = mdf.f_test(
            [p == main_var or p.startswith(main_var + '[')
             for p in mdf.params.index])
        # print(f_test)
        return f_test.pvalue
    return mixedlmp


def get_p_values(
    comp_table, meta_data, main_var, adj_formula, mixedlm_kwargs):
    # Calculate the p-value for each pairwise comparison of taxa.
    meta_data.index = comp_table.columns
    calc_p = _p_function(
        main_var, meta_data, adj_formula, mixedlm_kwargs)
    
    n_taxa = comp_table.shape[0]
    p_data = np.empty([n_taxa, n_taxa])
    p_data[:,:] = np.nan
    for i in range(n_taxa - 1):
        # Loop through each taxon.
        # For each taxon i, additive log ratio (alr) transform the 
        # OTU table using taxon i as the reference.
        # e.g. the first alr matrix will be the log abundance data 
        # (comp_table) recursively subtracted 
        # by the log abundance of 1st taxon (1st row) row-wisely,
        # and remove the first i rows since:
        # the first (i - 1) rows were calculated by previous
        # iterations, and
        # the i^th row contains all zeros.
        alr_data = comp_table.sub(comp_table.iloc[i], axis=1)
        # Here, we basically want to iteratively subtract each
        # row of the comp_table by its i^th column.
        alr_data = alr_data.iloc[i+1:]
        p_data[i+1:, i] = alr_data.apply(calc_p, 1)
    # Complete the p-value matrix.
    # What we got from above iterations is a lower triangle matrix 
    # of p-values.
    ix = np.logical_not(np.isnan(p_data.T))
    p_data[ix] = p_data.T[ix] 
    # set p-values on diagonal equal to 1
    p_data[np.eye(n_taxa, dtype=bool)] = 1
    return pd.DataFrame(
        p_data, index=comp_table.index, columns=comp_table.index)


def ANCOM(feature_table, meta_data, main_var, struc_zero=None,
          p_adj_method = 'fdr_bh', alpha=0.05, adj_formula=None,
          mixedlm_kwargs=None):
    # OTU table transformation: 
    # (1) Discard taxa with structural zeros (if any); 
    # (2) Add pseudocount (1) and take logarithm.
    if struc_zero is not None:
        num_struc_zero = struc_zero.apply(sum, axis=1)
        comp_table = feature_table.loc[num_struc_zero == 0]
    else:
        comp_table = feature_table
    comp_table = np.log(comp_table + 1)
    
    # Calculate the p-value for each pairwise comparison of taxa.
    p_data = get_p_values(
        comp_table, meta_data, main_var, adj_formula, mixedlm_kwargs)
    
    # Multiple comparisons correction.
    q_data = p_data.apply(
        lambda c: multipletests(c, method=p_adj_method)[1], axis=0)
    
    # Calculate the W statistic of ANCOM.
    # For each taxon, count the number of q-values < alpha.
    W = q_data.apply(lambda c: (c < alpha).sum(), axis=0)
    
    # Organize outputs
    # Declare a taxon to be differentially abundant based on the 
    # quantile of W statistic.
    # We perform (n_taxa - 1) hypothesis testings on each taxon, 
    # so the maximum number of rejections is (n_taxa - 1).
    n_taxa = len(W)
    out_comp = pd.DataFrame({
        'W': W,
        'detected_0.9': W > 0.9 * (n_taxa - 1),
        'detected_0.8': W > 0.8 * (n_taxa - 1),
        'detected_0.7': W > 0.7 * (n_taxa - 1),
        'detected_0.6': W > 0.6 * (n_taxa - 1),
    })
    
    # Taxa with structural zeros are automatically declared to
    # be differentially abundant
    if struc_zero is not None:
        out_comp.loc[num_struc_zero != 0] = (np.inf,) + (True,)*4

    return out_comp


def clr(x):
    tr = x.copy()
    nz = tr[(tr != 0.) & tr.notna()]
    nz = nz.apply(np.log)
    nz -= nz.mean()
    tr[(tr != 0.) & tr.notna()] = nz
    return tr


def ANCOM_volcano(feature_table, meta_data, main_var, W):
    # Draw volcano plot
    # Calculate clr
    clr_table = feature_table.apply(clr, axis=0)
    # Calculate clr mean difference
    def mean_difference(y):
        y.name = 'y'
        data = meta_data.join(y)
        formula = 'y ~ ' + main_var
        lm = ols(formula, data=data).fit()
        return lm.params[1:]
    eff_size = clr_table.apply(mean_difference, axis=1)

    

In [321]:
def test_get_p_values_manwhittneyu():
    feature_table = pd.read_csv('delivery-feature-table.tsv', sep=' ')
    meta_data = pd.read_csv("../data/ecam-sample-metadata.tsv", sep='\t')
    meta_data = meta_data[1:]
    meta_data = meta_data.rename(columns={'#SampleID': 'Sample.ID'})
    comp_table = np.log(feature_table + 1)
    
    main_var = 'delivery'
    adj_formula = None
    mixedlm_kwargs = None
    p_data = get_p_values(
        comp_table, meta_data, main_var, adj_formula, mixedlm_kwargs)
    
    answer = pd.read_csv('p-values-manwhittneyu.tsv', sep=' ')
    assert np.allclose(p_data, answer, rtol=1e-3)


def test_get_p_values_kruskalp():
    feature_table = pd.read_csv('delivery-feature-table.tsv', sep=' ')
    meta_data = pd.read_csv("../data/ecam-sample-metadata.tsv", sep='\t')
    meta_data = meta_data[1:]
    meta_data = meta_data.rename(columns={'#SampleID': 'Sample.ID'})
    comp_table = np.log(feature_table + 1)
    
    main_var = 'diet_3'
    adj_formula = None
    mixedlm_kwargs = None
    p_data = get_p_values(
        comp_table, meta_data, main_var, adj_formula, mixedlm_kwargs)
    
    answer = pd.read_csv('p-values-kruskalp.tsv', sep=' ')
    assert np.allclose(p_data, answer)


def test_get_p_values_anovap():
    feature_table = pd.read_csv('delivery-feature-table.tsv', sep=' ')
    meta_data = pd.read_csv("../data/ecam-sample-metadata.tsv", sep='\t')
    meta_data = meta_data[1:]
    meta_data = meta_data.rename(columns={'#SampleID': 'Sample.ID'})
    comp_table = np.log(feature_table + 1)
    
    main_var = 'delivery'
    adj_formula = 'studyid'
    mixedlm_kwargs = None
    p_data = get_p_values(
        comp_table, meta_data, main_var, adj_formula, mixedlm_kwargs)
    
    answer = pd.read_csv('p-values-anovap.tsv', sep=' ')
    assert np.allclose(p_data, answer)
    

def test_get_p_values_mixedlmp():
    feature_table = pd.read_csv('delivery-feature-table.tsv', sep=' ')
    meta_data = pd.read_csv("../data/ecam-sample-metadata.tsv", sep='\t')
    meta_data = meta_data[1:]
    meta_data = meta_data.rename(columns={'#SampleID': 'Sample.ID'})
    comp_table = np.log(feature_table + 1)
    
    main_var = 'delivery'
    adj_formula = None
    mixedlm_kwargs = {'groups': meta_data['studyid']}
    p_data = get_p_values(
        comp_table, meta_data, main_var, adj_formula, mixedlm_kwargs)
    
    answer = pd.read_csv('p-values-mixedlmp.tsv', sep=' ')
    assert np.allclose(p_data, answer, rtol=0.2, atol=0.03)
    

def test_ANCOM():
    feature_table = pd.read_csv('delivery-feature-table.tsv', sep=' ')
    meta_data = pd.read_csv("../data/ecam-sample-metadata.tsv", sep='\t')
    meta_data = meta_data[1:]
    meta_data = meta_data.rename(columns={'#SampleID': 'Sample.ID'})
    comp_table = np.log(feature_table + 1)
    
    main_var = 'diet_3'
    adj_formula = None
    mixedlm_kwargs = None
    p_adj_method = "fdr_bh"
    alpha = 0.05
    W = ANCOM(
        feature_table, meta_data, main_var,struc_zero, p_adj_method, 
        alpha, adj_formula, mixedlm_kwargs)
    
    answer = pd.read_csv('ancom-w.tsv', sep=' ')
    assert np.allclose(W['W'], answer['W'])
    assert (np.array(answer.iloc[:,2:]) 
            == np.array(W.iloc[:,1:])).all()

In [307]:
test_get_p_values_manwhittneyu()
test_get_p_values_kruskalp()
test_get_p_values_anovap()
test_get_p_values_mixedlmp() # go get a cup of tea
test_ANCOM()

In [270]:
# Step 1: Data preprocessing

feature_table = otu_data
sample_var = "Sample.ID"
group_var = "delivery"
out_cut = 0.05
zero_cut = 0.90
lib_cut = 0
neg_lb = True
prepro = feature_table_pre_process(
    feature_table, meta_data, sample_var, group_var, 
    out_cut, zero_cut, lib_cut, neg_lb)
feature_table = prepro['feature_table'] # Preprocessed feature table
meta_data = prepro['meta_data'] # Preprocessed metadata
struc_zero = prepro['structure_zeros'] # Structural zero info

/home/ben/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/home/ben/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: overflow encountered in true_divide
/home/ben/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
/home/ben/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in greater_equal
  del sys.path[0]


In [326]:
# Step 2: ANCOM

main_var = "delivery"
p_adj_method = "fdr_bh"
alpha = 0.05
adj_formula = None
#mixedlm_kwargs = {'groups': meta_data['studyid']}
mixedlm_kwargs = None
W = ANCOM(
    feature_table, meta_data, main_var,struc_zero, p_adj_method, 
    alpha, adj_formula, mixedlm_kwargs)

In [347]:
res = ANCOM_volcano(feature_table, meta_data, main_var, W)

,delivery[T.Vaginal]
feature-id,
k__Bacteria;__;__;__;__;__,-0.207527
k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__Actinomyces,0.238622
k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Micrococcaceae;g__Rothia,0.133285
k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Bifidobacteriales;f__Bifidobacteriaceae;g__Bifidobacterium,0.816852
k__Bacteria;p__Actinobacteria;c__Coriobacteriia;o__Coriobacteriales;f__Coriobacteriaceae;g__Collinsella,-0.123329
k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides,1.966017
k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Porphyromonadaceae;g__Parabacteroides,0.704957
k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Prevotellaceae;g__Prevotella,-0.266803
k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Rikenellaceae;g__,0.013308
